<a href="https://colab.research.google.com/github/HeqingZ/CorrectMviTv2/blob/main/MviTv2_Debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
#install detectron2
#the detectron2 package requires pyyaml version > 5.3 
!python -m pip install pyyaml==5.3.1
!pip install click
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 14.5 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44636 sha256=da9fbc160636f865ad1b752a0c5236c71341f4fa636785a3344602aad996426d
  Stored in directory: /root/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-dmwn3pam
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-dmwn3pam
   

In [3]:
#import some important utilities of Detectron2
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.12 ; cuda:  cu113
detectron2: 0.6


In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
#import features for the bbox
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

from detectron2.structures import BoxMode

# Load config from mvit repo
https://github.com/facebookresearch/detectron2/tree/main/projects/MViTv2


In [5]:
# Load original classification repo
!pip install timm
!git clone https://github.com/facebookresearch/mvit.git mvit
!curl -OL  https://github.com/facebookresearch/detectron2/blob/717ab9f0aeca216a2f800e43d705766251ba3a55/projects/MViTv2/configs/mask_rcnn_mvitv2_t_3x.pth # pretrained

# Load repo that related to detectron2

!python -m pip install -e mvit
# !cfg.python tools/main.py \
#   --cfg configs/MViTv2_T.yaml \
#   DATA.PATH_TO_DATA_DIR /content/drive/MyDrive/FLIR_THERMAL/train \
#   NUM_GPUS 8 \
#   TRAIN.BATCH_SIZE 512 \


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'mvit'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 130 (delta 61), reused 88 (delta 22), pack-reused 0
Receiving objects: 100% (130/130), 65.00 KiB | 16.25 MiB/s, done.
Resolving deltas: 100% (61/61), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  206k    0  206k    0     0   780k      0 --:--:-- --:--:-- --:--:--  780k
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mvit
     |████████████████████████████████| 130 kB 12.7 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=2d86318b6a6c8c93d9d414d0019a8796fede848f7730daa4748a7042

**MVIT-Detectron2 Codes =============================================**



In [6]:
!rm -r mvit_detectron2
!git clone https://github.com/jb2849/mvit_set.git mvit_detectron2

rm: cannot remove 'mvit_detectron2': No such file or directory
Cloning into 'mvit_detectron2'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 73 (delta 38), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.


**MVIT-Detectron2 Codes =============================================**

In [7]:
####
# Train Here
####
!python mvit_detectron2/lazyconfig_train_net.py --config-file mvit_detectron2/flir_config_3x.py --resume

{'backbone': {'bottom_up': {'embed_dim': 96, 'depth': 10, 'num_heads': 1, 'last_block_indexes': [0, 2, 7, 9], 'residual_pooling': True, 'drop_path_rate': 0.2, 'norm_layer': functools.partial(<class 'torch.nn.modules.normalization.LayerNorm'>, eps=1e-06), 'out_features': ['scale2', 'scale3', 'scale4', 'scale5'], '_target_': <class 'detectron2.modeling.backbone.mvit.MViT'>}, 'in_features': '${.bottom_up.out_features}', 'out_channels': 256, 'top_block': {'_target_': <class 'detectron2.modeling.backbone.fpn.LastLevelMaxPool'>}, '_target_': <class 'detectron2.modeling.backbone.fpn.FPN'>}, 'proposal_generator': {'in_features': ['p2', 'p3', 'p4', 'p5', 'p6'], 'head': {'in_channels': 256, 'num_anchors': 3, '_target_': <class 'detectron2.modeling.proposal_generator.rpn.StandardRPNHead'>}, 'anchor_generator': {'sizes': [[32], [64], [128], [256], [512]], 'aspect_ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64], 'offset': 0.0, '_target_': <class 'detectron2.modeling.anchor_generator.Default

In [8]:
### Testing

!python ./mvit_detectron2/lazyconfig_train_net.py --config-file ./mvit_detectron2/flir_config_3x.py --eval-only train.init_checkpoint=./drive/MyDrive/mvitv2Debug/model_final.pth

{'backbone': {'bottom_up': {'embed_dim': 96, 'depth': 10, 'num_heads': 1, 'last_block_indexes': [0, 2, 7, 9], 'residual_pooling': True, 'drop_path_rate': 0.2, 'norm_layer': functools.partial(<class 'torch.nn.modules.normalization.LayerNorm'>, eps=1e-06), 'out_features': ['scale2', 'scale3', 'scale4', 'scale5'], '_target_': <class 'detectron2.modeling.backbone.mvit.MViT'>}, 'in_features': '${.bottom_up.out_features}', 'out_channels': 256, 'top_block': {'_target_': <class 'detectron2.modeling.backbone.fpn.LastLevelMaxPool'>}, '_target_': <class 'detectron2.modeling.backbone.fpn.FPN'>}, 'proposal_generator': {'in_features': ['p2', 'p3', 'p4', 'p5', 'p6'], 'head': {'in_channels': 256, 'num_anchors': 3, '_target_': <class 'detectron2.modeling.proposal_generator.rpn.StandardRPNHead'>}, 'anchor_generator': {'sizes': [[32], [64], [128], [256], [512]], 'aspect_ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64], 'offset': 0.0, '_target_': <class 'detectron2.modeling.anchor_generator.Default